<a href="https://colab.research.google.com/github/salmabenhassin/CognoRise-Infotech-Projects/blob/main/Copie_de_Translate_EN_to_FR_(Tensorflow_%26_Transformers).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'language-translation-englishfrench:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F592212%2F1067156%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240911%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240911T112600Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6262d4056c2f4378db84f52e3b21e958bee3e0494ca73b72f9e2dd942f604a6d01ecc5c5c962ebe398247cdb8ebc68626b203584f66b12f4883e9c321e6b943770f6bfeac705dda3f53e37cbee6aecc8c6e684830e0f629f39f058b56142667956c37daf7f98270f8795b4d27634ae9d70fa6fe273c3910ec168eac26926a1e4ed12f816153a577ef5a4d18c5d8685feb6187e8b49135c28c8f7dcbd87552f8e2bcf62bc32cdfb4b3c05287bb74463445aeb78831fef4dcaefce55d0d48a3d3757ffd2371030e274c6c04cb1d13c361ede7d5752a7c2dad12a6078cfb684db5866058689b75c3fba1b67a19ac93691c8ca44ad1506e498c0bcbfbc9fabdc3bd0,english-to-german:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F972382%2F1644506%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240911%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240911T112600Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D14f8d5ceda913ae9d612e94d82162544672252d77f3eae28027d703577e845b03f7770559e2cffa8eb6455a373be30b2652ff731b83f1d831eb024564f2216c0ca12194499f2ec74e087ef711dbe5d8223271b38f396ccf9c2541a22b8f0099dc82c4f6bc91362bc4ee299cc3fd49df278a125912f67708c12990abb7059d658eec48f886c977021729b1a39b80d4bdf214eaa8cc75d1aeaa77f1ab6027f562f7d1de5dc8740e7df96c01ccdf98d27426fd3b777bca0bac73a7f13e29d8c9ea37f09851d57337369455d679b4cacb0505d0ad9f88e504dc33cd30d292526eff12a3152f8fe44d5d041d1bb58a6266630d4ecc19ec51caf863bb108f3d2483e8d,en-fr-translation-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1148896%2F1926230%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240911%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240911T112600Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D31b806613eda6a7baa00b80cf2d975a93a34c541f4eef0c86bb0ecff8b1b02806939cec30d1eb1dac5304f87c08338819805fff66c7ef91e2ec586bb68d56af5ffe12d6f6ac1511ff3ba8096b525ff099ba05f9914eca8b4fee2c2c90bf77332ea03d9ac8981c318349d1896fc1a68efa1c849f8ac8438f7cce73bb1f13639bdc96401b598b65dd99ff5afef85d02e0229b713ae8862a625b3c06938af4b608211562787429e0a74e9a99c01def3000128a3fa42edc25371e261bdf8f5a62a685e25cf8d1f1bf03f451b45325cc87553f34ce576f6a3b41acd11b93ef576b2b9e7e188d9bb64da774ff80c62171e961cb9c6ea901c3caad779dc50e1c51e63a5,english-french-translation:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2326821%2F4165279%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240911%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240911T112600Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D7f70fea263cb07064c6bf92fba44098534741e71bbb97fac7c90fc6a0c2990c4eb0b3ce04a44183050881f6c99284127f68c288d4a11d5451d18823d7f57563210ddbddb6065c2bc8a53127bbcb2219c5ae83f7a4262af8ff59cab5a6c35cf184c2a55217f8492c1115e2074cf9d8168d14ecb6d57643f87fbd95f5d85f675bacbe855fc594928fe983c61d7bde85796e1a10ccf55b2517bd97c8ec2a6decefd962800f9dda355a35b86a31199f6b8f808f930692b5a9cb281ed09120dc1d6a04254e3e5e528ff88e0d9fbaa96c0c16cd5ee667beb743dcb9fdc7a4744bae5985ed236f05a627b6c133c1a6c89aee721a608215046b2a71c8df811221eb67179'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
#import tensorflow as tf

# Détecter et initialiser le TPU
#tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# Vérifier si le TPU a été détecté
#if tpu.cluster_spec():
#    print("Connexion au TPU réussie.")
#    print("Nom du TPU :", tpu.get_master())
#    print("Adresse du TPU :", tpu.get_master())
#else:
#    print("Impossible de se connecter au TPU.")

# Instancier la stratégie de distribution TPU
#tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
#print(tpu_strategy)

# **Let's try with a custom model**

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd

# Chargement des données
data = pd.read_csv("/kaggle/input/language-translation-englishfrench/eng_-french.csv")
english_sentences = data["English words/sentences"].tolist()
french_sentences = data["French words/sentences"].tolist()
print(english_sentences[:5])

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

['Hi.', 'Run!', 'Run!', 'Who?', 'Wow!']


In [ ]:
# Adaptation des tokenizers aux données
tokenizer_eng = Tokenizer()
tokenizer_eng.fit_on_texts(english_sentences)
eng_seq = tokenizer_eng.texts_to_sequences(english_sentences)

tokenizer_fr = Tokenizer()
tokenizer_fr.fit_on_texts(french_sentences)
fr_seq = tokenizer_fr.texts_to_sequences(french_sentences)

# Utilisation du nombre de mots dans le tokenizer pour définir l'embedding
vocab_size_eng = len(tokenizer_eng.word_index) + 1
vocab_size_fr = len(tokenizer_fr.word_index) + 1

# Padding
max_length = max(len(seq) for seq in eng_seq + fr_seq)
eng_seq_padded = pad_sequences(eng_seq, maxlen=max_length, padding='post')
fr_seq_padded = pad_sequences(fr_seq, maxlen=max_length, padding='post')


In [ ]:
embedding_dim = 256
units = 512

# Encoder
encoder_inputs = Input(shape=(max_length,))
enc_emb = Embedding(input_dim=vocab_size_eng, output_dim=embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(units, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_length,))
dec_emb_layer = Embedding(input_dim=vocab_size_fr, output_dim=embedding_dim)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(vocab_size_fr, activation='softmax')
output = decoder_dense(decoder_outputs)

# Modèle
model = Model([encoder_inputs, decoder_inputs], output)

# Compilation du modèle
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(eng_seq_padded, fr_seq_padded, test_size=0.2)
model.fit([X_train, X_train], y_train, validation_data=([X_val, X_val], y_val), epochs=5, batch_size=32)

Epoch 1/5
4391/4391 [==============================] - 550s 124ms/step - loss: 0.7006 - accuracy: 0.9005 - val_loss: 0.5546 - val_accuracy: 0.9105
Epoch 2/5
4391/4391 [==============================] - 537s 122ms/step - loss: 0.4871 - accuracy: 0.9143 - val_loss: 0.4691 - val_accuracy: 0.9171
Epoch 3/5
4391/4391 [==============================] - 537s 122ms/step - loss: 0.3942 - accuracy: 0.9214 - val_loss: 0.4357 - val_accuracy: 0.9211
Epoch 4/5
4391/4391 [==============================] - 536s 122ms/step - loss: 0.3363 - accuracy: 0.9270 - val_loss: 0.4230 - val_accuracy: 0.9233
Epoch 5/5
4391/4391 [==============================] - 537s 122ms/step - loss: 0.2973 - accuracy: 0.9319 - val_loss: 0.4184 - val_accuracy: 0.9249


In [ ]:
model.save("seq2seq_translation_v3.h5")

In [ ]:
#model = tf.keras.models.load_model("seq2seq_translation_v3.h5")

In [ ]:
def translate_sentence(sentence):
    seq = tokenizer_eng.texts_to_sequences([sentence])
    padded = pad_sequences(seq, maxlen=max_length, padding='post')
    translated = np.argmax(model.predict([padded, padded]), axis=-1)

    translated_sentence = []
    for i in translated[0]:
        if i in tokenizer_fr.index_word:
            translated_sentence.append(tokenizer_fr.index_word[i])
        else:
            translated_sentence.append(' ')  # Token inconnu si l'indice n'est pas trouvé dans le tokenizer

    return ' '.join(translated_sentence)

input_sentence = "I am tired ."
translated_sentence = translate_sentence(input_sentence)
print(f"Input: {input_sentence}")
print(f"Translated: {translated_sentence}")


1/1 [==============================] - 1s 1s/step
Input: I am tired .
Translated: je suis fatigué                                                                                                        


# **Transformers**

In [ ]:
!pip install transformers

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch

# Chargez le modèle T5 pré-entraîné pour la traduction anglais-français
model_name = "t5-base"
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Chargez le tokenizer associé
tokenizer = T5Tokenizer.from_pretrained(model_name)

def translate_english_to_french(english_sentence):
    # Préparez la phrase pour l'entrée du modèle (ajoutez le préfixe "translate English to French: ")
    input_text = "translate English to French: " + english_sentence

    # Tokenize la phrase et encode les tokens en IDs
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # Générez la traduction en français
    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids)

    # Décodage de la traduction en français à partir des IDs de tokens générés
    french_translation = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return french_translation

english_sentence = "Hello, how was your day ?"
french_translation = translate_english_to_french(english_sentence)

print("English:", english_sentence)
print("French:", french_translation)

/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -

English: Hello, how was your day ?
French: Bonjour, quelle a été votre journée?


In [ ]:
english_sentence = "hello ?"
french_translation = translate_english_to_french(english_sentence)

print("English:", english_sentence)
print("French:", french_translation)

English: hello ?
French: Bonjour?


We got better result with transformers models. Custom models help us to understand how it work.